In [2]:
import sys
sys.path.append("../../dataprocessing")
import numpy as np
import os
import PIL
import PIL.Image
from PIL import ImageDraw
import tensorflow as tf
# import tensorflow_datasets as tfds
import pandas as pd
from tqdm import tqdm
import random
import data_process
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from scipy.spatial import distance

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, plot_confusion_matrix
import datetime as dt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from xgboost import XGBClassifier

## 함수 정의

In [3]:
cd ../

c:\Users\jongm\Desktop\MINI\projects\Facial-Expression-Recognition\modeling


In [4]:
def cal_dist(data):
    landmark_matrixs, landmark_matrixs_df = [], []
    lp = [point[:-1] for point in data.columns if '-' in point]
    set_lp = sorted(set(lp), key = lambda x: lp.index(x))
    for idx, row in data.iterrows():
        # print(str(idx)+" 하는중")
        temp_list = []
        for i in range(len(set_lp)):
            temp_list.append([row[set_lp[i]+'x'], row[set_lp[i]+'y']])
        temp = pd.DataFrame(distance.cdist(temp_list, temp_list, metric='euclidean'), index=set_lp, columns=set_lp)
        landmark_matrixs.append(temp)
        landmark_matrixs_df.append(temp.to_numpy()[np.triu_indices(len(set_lp), 1)])
    return landmark_matrixs, pd.DataFrame(landmark_matrixs_df)

In [5]:
def show_img(data, path, landmark_points):    
    img_array = np.fromfile(path, np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    # img = cv2.imread(path, cv2.IMREAD_COLOR)    
    drawing_image = img.copy()
    target_landmark = landmark_points.index
    # print(landmark_points.index)
    # print(data)
    for idx in range(0,len(data.columns),2):
        radius = 2
        x = data.loc[int(float(path.split('/')[4][:-4]))][data.columns[idx]]
        y = data.loc[int(float(path.split('/')[4][:-4]))][data.columns[idx+1]]
        # if data.columns[idx] == 'frame':
        #     continue
        if data.columns[idx][:-1] in target_landmark:
            # print(x,y)
            cv2.circle(drawing_image, center= tuple([int(x),int(y)]), radius=radius*2, color=(255,255,0), thickness=-1, lineType=cv2.LINE_AA)
        else:
            cv2.circle(drawing_image, center= tuple([int(x),int(y)]), radius=radius, color=(0,255,0), thickness=-1, lineType=cv2.LINE_AA)
    # img_show(drawing_image)
    left = data.loc[int(float(path.split('/')[4][:-4]))]['left']
    top = data.loc[int(float(path.split('/')[4][:-4]))]['top']
    right = data.loc[int(float(path.split('/')[4][:-4]))]['right']
    bottom = data.loc[int(float(path.split('/')[4][:-4]))]['bottom']
    plt.subplots(figsize=(10,10))
    # cv2.rectangle(drawing_image, (int(left), int(top)), (int(right), int(bottom)), (255, 255, 0), 3)
    plt.imshow(cv2.cvtColor(drawing_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')

In [6]:
def show_diff(*args):
    data = args[0]
    window = args[1]
    erase_list = ['frame','left','top','right','bottom','time_diff']
    data3 = data[[point for point in data.columns if '-' in point] + erase_list]

    if len(args) == 3:
        idxs = args[2]
        i = 0
        for idx in idxs:
            if i == 0:
                data2 = data[[point for point in data.columns if str(idx) == point.split('-')[0]]]
            else:
                data2 = pd.concat([data2, data[[point for point in data.columns if str(idx) == point.split('-')[0]]]], axis=1)
            i += 1
        data2 = pd.concat([data2, data[erase_list]], axis=1)
        # print(data2.columns)
    else:
        data2 = data3.copy()

    # print(list(set(data.columns) - set(erase_list)))
    remain_list = sorted(list(set(data2.columns) - set(erase_list)))
    diff_df = pd.DataFrame(columns=list(range(len(data2))), index=list(set([c[:-1] for c in remain_list])))

    # print(data2, remain_list)    
    for row in range(0, len(remain_list), 2):
        init_point = [data2.iloc[0][remain_list[row]], data2.iloc[0][remain_list[row+1]]]
        # print(remain_list[col], remain_list[col+1])
        temp2 = 0.0
        # diff_df[remain_list[col][:-1]] = 0
        for col in range(len(data2)):    
            # print(init_point, data.loc[col][remain_list[row]], data.loc[col][remain_list[row+1]])    
            temp = distance.euclidean(init_point, [data2.loc[col][remain_list[row]], data2.loc[col][remain_list[row+1]]])
            temp2 += temp
            # for start in range(0,len(data.index), window):
            #     # print(max(data.iloc[start:start+5][col]) - min(data.iloc[start:start+5][col]), max(data.iloc[start:start+5][col]), min(data.iloc[start:start+5][col]))
            #     temp += max(data.iloc[start:start+5][col]) - min(data.iloc[start:start+5][col])
            diff_df.at[remain_list[row][:-1], col] = temp
        diff_df.at[remain_list[row][:-1], 'sum'] = temp2
        # print(remain_list[col] + ", "+ remain_list[col+1])
    return diff_df.sort_values('sum', ascending=False), data3

In [7]:
def make_flow_chart(data, diff_df, part, order, image_order, path):
    sns.set_style("darkgrid")
    fig, ax = plt.subplots(figsize = (20,10))
    # sort = diff_df.iloc[:order]
    diff_df = diff_df.drop(['sum'], axis=1)
    # print(diff_df)
    new_data = diff_df.iloc[:order]
    print(new_data.index)
    # new_data = new_data[[point for point in new_data.columns if str(part) == point.split('-')[0]]]
    temp_plot = sns.lineplot(data = new_data.transpose(), markers=False, dashes=False)  
    # print(path+str(image_order) + '.0.jpg')
    show_img(data, path + str(image_order) + '.0.jpg', new_data)
    # ax[idx, col].set_title(str(idx) +" 번 클러스터")
    # if col==0:        
    #     ax[idx, col].set_ylabel(list(custom_sort.keys())[idx])
    #     ax[idx, col].yaxis.label.set_size(20)
    # ax[idx].set_ylabel(list(custom_sort.keys())[idx] + " group")
    # ax[idx].set_xticks(list(range(1,len(seq)+1)))        
    # for row in data.iterrows():
    #     ax[idx].text(row[0], int(row[1]['quiz']), int(row[1]['quiz']), fontsize=12, ha='center') 
    #     ax[idx].text(row[0], int(row[1]['proj']), int(row[1]['proj']), fontsize=12, ha='center')

In [78]:
def predict(data_path1, data_path2, model):
    landmarks = pd.read_csv(data_path1)
    landmarks2 = pd.read_csv(data_path2)
    landmark_matrixs, landmark_matrixs_df = cal_dist(landmarks)
    landmark_matrixs_df['class'] = 0
    landmark_matrixs2, landmark_matrixs_df2 = cal_dist(landmarks2)
    landmark_matrixs_df2['class'] = 1
    df = pd.concat([landmark_matrixs_df, landmark_matrixs_df2], axis=0)
    length_i = len(landmark_matrixs[0].columns)
    start = 1
    name_list = []
    for i in landmark_matrixs[0].index:
        for j in range(start, length_i):
            name_list.append(str(i)+' ↔ '+str(landmark_matrixs[0].columns[j]))
        start += 1

    X = df.drop(['class'], axis=1)
    X = X.astype('float')
    y = df['class']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

    if model == 'rf':
        set_model = RandomForestClassifier(criterion='entropy', bootstrap=True, random_state=2022, oob_score=True)
    elif model == 'xgboost':
        set_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

    set_model.fit(X_train, y_train)
    pred = set_model.predict(X_test)
    print(confusion_matrix(y_test, pred))
    # accuracy = accuracy_score(y_test, pred)
    # print('랜덤 포레스트 정확도: {:.4f}'.format(accuracy))
    print('훈련세트 정확도: {:.3f}' .format(set_model.score(X_train, y_train)))
    print('테스트세트 정확도: {:.3f}' .format(set_model.score(X_test, y_test)))
    importance = pd.DataFrame({'landmarks': name_list, 'importance': set_model.feature_importances_})
    importance = importance.sort_values('importance',ascending=False)
    importance = importance.astype({'landmarks':'str'})
    print(importance)

> 0은 얼굴 테두리 \
> 1, 2는 왼쪽 눈썹 위, 아래 \
> 3, 4는 오른쪽 눈썹 위, 아래 \
> 5, 6은 왼쪽, 오른쪽 눈 \
> 7, 8은 위쪽 입술 윗부분, 아래부분 \
> 9, 10은 아래쪽 입술 윗부분, 아래부분 \
> 11은 코 세로축 \
> 12는 코 아래부분

## 1. 눈썹 들어올리기

In [79]:
predict('../data/images/중립/202209.03105902855.csv', '../data/images/눈썹들어올리기+1/202209.06220402854.csv', 'rf')

[[11  0]
 [ 0 11]]
훈련세트 정확도: 1.000
테스트세트 정확도: 1.000
        landmarks  importance
2235  0-18 ↔ 0-31        0.01
4208   1-0 ↔ 10-6        0.01
7995   7-4 ↔ 12-2        0.01
6816  5-13 ↔ 10-7        0.01
262    0-1 ↔ 14-0        0.01
...           ...         ...
2943   0-24 ↔ 6-4        0.00
2942   0-24 ↔ 6-3        0.00
2941   0-24 ↔ 6-2        0.00
2940   0-24 ↔ 6-1        0.00
8777  13-0 ↔ 14-0        0.00

[8778 rows x 2 columns]


In [80]:
predict('../data/images/중립/202209.03105902855.csv', '../data/images/눈썹들어올리기+1/202209.06220402854.csv', 'xgboost')

[[11  0]
 [ 0 11]]
훈련세트 정확도: 1.000
테스트세트 정확도: 1.000
        landmarks  importance
5       0-0 ↔ 0-6         1.0
0       0-0 ↔ 0-1         0.0
5855    5-0 ↔ 5-4         0.0
5849   4-4 ↔ 12-2         0.0
5850   4-4 ↔ 13-0         0.0
...           ...         ...
2921   0-24 ↔ 4-3         0.0
2920   0-24 ↔ 4-2         0.0
2919   0-24 ↔ 4-1         0.0
2918   0-24 ↔ 4-0         0.0
8777  13-0 ↔ 14-0         0.0

[8778 rows x 2 columns]


## 3. 두 눈 감기

In [81]:
predict('../data/images/중립/202209.03105902855.csv', '../data/images/두눈감기+1/202209.03105914019.csv', 'rf')

[[13  0]
 [ 0 12]]
훈련세트 정확도: 1.000
테스트세트 정확도: 1.000
        landmarks  importance
2103  0-16 ↔ 12-0    0.047749
2090   0-16 ↔ 9-7    0.024804
1987  0-15 ↔ 12-0    0.022706
1870  0-14 ↔ 12-0    0.018521
1761  0-14 ↔ 0-19    0.018284
...           ...         ...
2983   0-24 ↔ 9-8    0.000000
2982   0-24 ↔ 9-7    0.000000
2981   0-24 ↔ 9-6    0.000000
2980   0-24 ↔ 9-5    0.000000
8777  13-0 ↔ 14-0    0.000000

[8778 rows x 2 columns]


In [83]:
predict('../data/images/중립/202209.03105902855.csv', '../data/images/두눈감기+1/202209.03105914019.csv', 'xgboost')

[[13  0]
 [ 0 12]]
훈련세트 정확도: 1.000
테스트세트 정확도: 1.000
        landmarks  importance
1987  0-15 ↔ 12-0    0.482296
2103  0-16 ↔ 12-0    0.195566
1644  0-13 ↔ 0-20    0.160161
564     0-4 ↔ 3-1    0.055626
1761  0-14 ↔ 0-19    0.042627
...           ...         ...
2927   0-24 ↔ 5-4    0.000000
2926   0-24 ↔ 5-3    0.000000
2925   0-24 ↔ 5-2    0.000000
2924   0-24 ↔ 5-1    0.000000
8777  13-0 ↔ 14-0    0.000000

[8778 rows x 2 columns]


## 4. 두 눈 크게 뜨기

In [84]:
predict('../data/images/중립/202209.03105902855.csv', '../data/images/두눈크게뜨기+1/202209.03152411046.csv', 'rf')

[[13  1]
 [ 0 13]]
훈련세트 정확도: 1.000
테스트세트 정확도: 0.963
        landmarks  importance
3813  0-32 ↔ 10-1        0.01
4483    1-3 ↔ 9-8        0.01
2357   0-19 ↔ 1-4        0.01
2362   0-19 ↔ 2-4        0.01
4559    1-4 ↔ 8-1        0.01
...           ...         ...
2930   0-24 ↔ 5-7        0.00
2929   0-24 ↔ 5-6        0.00
2928   0-24 ↔ 5-5        0.00
2927   0-24 ↔ 5-4        0.00
8777  13-0 ↔ 14-0        0.00

[8778 rows x 2 columns]


In [85]:
predict('../data/images/중립/202209.03105902855.csv', '../data/images/두눈크게뜨기+1/202209.03152411046.csv', 'xgboost')

[[12  2]
 [ 2 11]]
훈련세트 정확도: 1.000
테스트세트 정확도: 0.852
        landmarks  importance
16     0-0 ↔ 0-17         1.0
0       0-0 ↔ 0-1         0.0
5847   4-4 ↔ 12-0         0.0
5849   4-4 ↔ 12-2         0.0
5850   4-4 ↔ 13-0         0.0
...           ...         ...
2921   0-24 ↔ 4-3         0.0
2920   0-24 ↔ 4-2         0.0
2919   0-24 ↔ 4-1         0.0
2918   0-24 ↔ 4-0         0.0
8777  13-0 ↔ 14-0         0.0

[8778 rows x 2 columns]


## 9. 입벌리기

In [86]:
predict('../data/images/중립/202209.03105902855.csv', '../data/images/입벌리기+1/202209.05132412050.csv', 'rf')

[[12  0]
 [ 0 11]]
훈련세트 정확도: 1.000
테스트세트 정확도: 1.000
        landmarks  importance
38      0-0 ↔ 1-3        0.02
4559    1-4 ↔ 8-1        0.01
3961  0-34 ↔ 5-13        0.01
467     0-3 ↔ 6-6        0.01
464     0-3 ↔ 6-3        0.01
...           ...         ...
2940   0-24 ↔ 6-1        0.00
2939   0-24 ↔ 6-0        0.00
2938  0-24 ↔ 5-15        0.00
2937  0-24 ↔ 5-14        0.00
8777  13-0 ↔ 14-0        0.00

[8778 rows x 2 columns]


In [87]:
predict('../data/images/중립/202209.03105902855.csv', '../data/images/입벌리기+1/202209.05132412050.csv', 'xgboost')

[[12  0]
 [ 0 11]]
훈련세트 정확도: 1.000
테스트세트 정확도: 1.000
        landmarks  importance
8       0-0 ↔ 0-9         1.0
0       0-0 ↔ 0-1         0.0
5855    5-0 ↔ 5-4         0.0
5849   4-4 ↔ 12-2         0.0
5850   4-4 ↔ 13-0         0.0
...           ...         ...
2921   0-24 ↔ 4-3         0.0
2920   0-24 ↔ 4-2         0.0
2919   0-24 ↔ 4-1         0.0
2918   0-24 ↔ 4-0         0.0
8777  13-0 ↔ 14-0         0.0

[8778 rows x 2 columns]


## 10. 입술 오므리기

In [88]:
predict('../data/images/중립/202209.03105902855.csv', '../data/images/입술오므리기+1/202209.05132437930.csv', 'rf')

[[10  0]
 [ 0  9]]
훈련세트 정확도: 1.000
테스트세트 정확도: 1.000
        landmarks  importance
38      0-0 ↔ 1-3        0.02
7138    6-3 ↔ 7-1        0.01
4853   2-2 ↔ 10-7        0.01
483     0-3 ↔ 7-6        0.01
482     0-3 ↔ 7-5        0.01
...           ...         ...
2939   0-24 ↔ 6-0        0.00
2938  0-24 ↔ 5-15        0.00
2937  0-24 ↔ 5-14        0.00
2936  0-24 ↔ 5-13        0.00
8777  13-0 ↔ 14-0        0.00

[8778 rows x 2 columns]


In [89]:
predict('../data/images/중립/202209.03105902855.csv', '../data/images/입술오므리기+1/202209.05132437930.csv', 'xgboost')

[[10  0]
 [ 0  9]]
훈련세트 정확도: 1.000
테스트세트 정확도: 1.000
        landmarks  importance
2       0-0 ↔ 0-3    0.938667
1       0-0 ↔ 0-2    0.061333
5854    5-0 ↔ 5-3    0.000000
5848   4-4 ↔ 12-1    0.000000
5849   4-4 ↔ 12-2    0.000000
...           ...         ...
2920   0-24 ↔ 4-2    0.000000
2919   0-24 ↔ 4-1    0.000000
2918   0-24 ↔ 4-0    0.000000
2917   0-24 ↔ 3-4    0.000000
8777  13-0 ↔ 14-0    0.000000

[8778 rows x 2 columns]


## 16. 미소짓기

In [90]:
predict('../data/images/중립/202209.03105902855.csv', '../data/images/미소짓기+1/202209.05132448886.csv', 'rf')

[[11  0]
 [ 0 10]]
훈련세트 정확도: 1.000
테스트세트 정확도: 1.000
        landmarks  importance
8164    7-9 ↔ 9-6        0.02
38      0-0 ↔ 1-3        0.02
483     0-3 ↔ 7-6        0.01
5079    3-0 ↔ 7-1        0.01
3732   0-32 ↔ 1-1        0.01
...           ...         ...
2935  0-24 ↔ 5-12        0.00
2934  0-24 ↔ 5-11        0.00
2933  0-24 ↔ 5-10        0.00
2932   0-24 ↔ 5-9        0.00
8777  13-0 ↔ 14-0        0.00

[8778 rows x 2 columns]


In [91]:
predict('../data/images/중립/202209.03105902855.csv', '../data/images/미소짓기+1/202209.05132448886.csv', 'xgboost')

[[11  0]
 [ 1  9]]
훈련세트 정확도: 1.000
테스트세트 정확도: 0.952
        landmarks  importance
6       0-0 ↔ 0-7         1.0
0       0-0 ↔ 0-1         0.0
5855    5-0 ↔ 5-4         0.0
5849   4-4 ↔ 12-2         0.0
5850   4-4 ↔ 13-0         0.0
...           ...         ...
2921   0-24 ↔ 4-3         0.0
2920   0-24 ↔ 4-2         0.0
2919   0-24 ↔ 4-1         0.0
2918   0-24 ↔ 4-0         0.0
8777  13-0 ↔ 14-0         0.0

[8778 rows x 2 columns]
